# **1. Configuración del Ambiente**


---

In [71]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

# **2. Creación de Modelo de Regresión Lineal**


---

In [72]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visitNumber         12283 non-null  int64  
 1   browser             12283 non-null  int64  
 2   operatingSystem     12283 non-null  int64  
 3   deviceCategory      12283 non-null  int64  
 4   continent           12283 non-null  int64  
 5   country             12283 non-null  int64  
 6   city                12283 non-null  int64  
 7   networkDomain       12283 non-null  int64  
 8   source              12283 non-null  int64  
 9   medium              12283 non-null  int64  
 10  keyword             12283 non-null  int64  
 11  referralPath        12283 non-null  int64  
 12  adContent           12283 non-null  int64  
 13  pageviews           12283 non-null  int64  
 14  bounces             12283 non-null  float64
 15  newVisits           12283 non-null  float64
 16  tran

In [73]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred = linear_model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 327.96039211735996


In [74]:
#definiendo el K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

In [75]:
accuracy_cross_val = cross_val_score(estimator = linear_model,
                                       X = X_train,
                                       y = y_train,
                                       scoring = 'neg_mean_squared_error',
                                       cv = cv)
print('Accuracy promedio de mse: {}'.format(round(accuracy_cross_val.mean(), 3)))

Accuracy promedio de mse: -303.639


In [76]:
# Selección de hiperparámetros para la regresión lineal
param_linear_regression = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1, None],
    'positive': [True, False]
}

In [77]:
random_cv_linear = RandomizedSearchCV(
    linear_model,
    param_linear_regression,
    n_iter=5,
    cv=cv,
    scoring='neg_mean_squared_error',  # Corrección del scoring a 'neg_mean_squared_error'
    n_jobs=1
)

In [78]:
random_cv_linear.fit(X_train, y_train)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
                   estimator=LinearRegression(), n_iter=5, n_jobs=1,
                   param_distributions={'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'n_jobs': [-1, None],
                                        'positive': [True, False]},
                   scoring='neg_mean_squared_error')

In [79]:
#la mejor combinación
random_cv_linear.best_params_

{'positive': False, 'n_jobs': None, 'fit_intercept': False, 'copy_X': False}

In [80]:
#el mejor score
random_cv_linear.best_score_

-305.2043962495942

In [81]:
# Crear una nueva instancia de LinearRegression con los mejores parámetros encontrados
best_linear_model = LinearRegression(**random_cv_linear.best_params_)

# Entrenar el modelo con los datos de entrenamiento
best_linear_model.fit(X_train, y_train)

y_pred = best_linear_model.predict(X_test)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 327.9415904645853
R^2 Score: 0.23627431852306613
Root Mean Squared Error: 18.109157640944687


In [82]:
model_scores = {
    'model': 'LinearRegression-Edwin6',
    'mse': mse,
    'r2': r2,
    'rmse': rmse
}
print(model_scores)

{'model': 'LinearRegression-Edwin6', 'mse': 327.9415904645853, 'r2': 0.23627431852306613, 'rmse': 18.109157640944687}


In [83]:
def save_scores(model_scores: dict):
  status = []
  #Utilizamos un try-except en caso de que el archivo no exista que cree uno
  try:
    #Si el archivo existe continúa por acá
    scores = pd.read_csv('scores.csv', sep=';')
    status.append('El archivo existe')
    exist = model_scores['model'] in scores['model'].values
    #Validamos si el modelo ya existe en el archivo de scores
    if (exist):
      #Si el modelo ya existe reemplazamos sus valores
      status.append('El modelo existe')
      criteria = scores['model'] == model_scores['model']
      index = scores[criteria].index[0]
      scores.iloc[index] = model_scores
      status.append('Se reemplazaron los valores del modelo')
    else:
      #Si el modelo no existe lo agregamos
      status.append('El modelo no existe')
      df_model_scores = pd.DataFrame(model_scores, index=[0])
      scores = pd.concat([scores, df_model_scores], ignore_index=True)
      status.append('Se añadió modelo nuevo y sus valores')
  except:
    #Si no existe el archivo lo creamos y cargamos los scores del modelo
    status.append('El archivo no existe')
    scores = pd.DataFrame(model_scores, index=[0])

  status.append('Se sobrescrive el archivo scores.csv con valores nuevos')
  scores.to_csv('scores.csv', sep=';', index=False)
  return status

In [84]:
save_scores(model_scores)

['El archivo existe',
 'El modelo existe',
 'Se reemplazaron los valores del modelo',
 'Se sobrescrive el archivo scores.csv con valores nuevos']